In [47]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import jaccard_score
from sklearn.metrics import make_scorer
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.ensemble import MajorityVotingClassifier
from skmultilearn.cluster import FixedLabelSpaceClusterer
from skmultilearn.ensemble import RakelO
from skmultilearn.model_selection import IterativeStratification
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier

from mlxtend.evaluate.time_series import GroupTimeSeriesSplit
from mlxtend.evaluate.time_series import plot_splits
from mlxtend.evaluate.time_series import print_cv_info
from mlxtend.evaluate.time_series import print_split_info
import optuna
from functools import partial
from collected_functions import *

Set base variable values

In [28]:
random_state = 42
np.random.seed(random_state)


In [57]:
def display_label_cardinality_error(
        y_true: pd.DataFrame,
        y_pred: np.ndarray,
    ) ->None:
    '''
    Create a scatterplot from y_true and y_pred to show accuracy of number of labels predicted.
    Created with the help of Google Gemini
    '''
    if not isinstance(y_pred, np.ndarray):
        y_pred = y_pred.toarray()

    # 1. Calculate Cardinalities (Sum across the label axis, axis=1)
    true_cardinality = np.sum(y_true, axis=1)
    pred_cardinality = np.sum(y_pred, axis=1)

    # 2. Calculate Hamming Loss per sample
    # Hamming loss for sample i = (number of mismatches) / N_LABELS
    mismatches = np.sum(y_true != y_pred, axis=1)
    hamming_loss_Score_ = mismatches / y_true.shape[1]

    # 3. Create the Visualization DataFrame
    df_error_vis = pd.DataFrame({
        'Sample_ID': [f'Sample_{i}' for i in range(y_true.shape[0])],
        'True_Cardinality': true_cardinality,
        'Predicted_Cardinality': pred_cardinality,
        'Hamming_Loss_Score_': hamming_loss_Score_,
        'Total_Errors': mismatches
    })

    fig = px.scatter(
        df_error_vis,
        x='True_Cardinality',
        y='Predicted_Cardinality',
        # Color the points by the error metric
        color='Hamming_Loss_Score_',
        # Scale the size of the points by the number of errors
        # size='Total_Errors',
        hover_data=['Sample_ID', 'Total_Errors'],
        title='Multilabel Error Analysis: Predicted vs. True Label Cardinality',
        color_continuous_scale=px.colors.sequential.Plasma, # Use a good sequential color scale
    )

    # Set axis limits and labels for a clear square comparison
    max_cardinality = y_true.shape[1]
    fig.update_xaxes(
        range=[-0.5, max_cardinality + 0.5],
        tickvals=np.arange(0, max_cardinality + 1),
        title="True Label Cardinality (Complexity)"
    )
    fig.update_yaxes(
        range=[-0.5, max_cardinality + 0.5],
        tickvals=np.arange(0, max_cardinality + 1),
        title="Predicted Label Cardinality (Model Output)"
    )

    # Add a diagonal line for perfect cardinality prediction (where X=Y)
    fig.add_shape(
        type="line", line=dict(dash='dash', color='gray'),
        x0=0, y0=0, x1=max_cardinality, y1=max_cardinality
    )

    fig.show()

Adjust Pandas settings to display more information before truncation.

In [14]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

Load combined MA and DDS training data

In [15]:
df = pd.read_parquet('combined_training_set.parquet')
print(df.shape)
df.head()

(6566, 21)


,incidentId,state,incidentTypeMA,region,maType,maPriority,supportFunction,agencyId,maId,declarationType,assistanceRequested,statementOfWork,femaDeclarationString,incidentType,incidentBeginDate,fipsStateCode,designatedIncidentTypes,declarationTitle,year,month,day
45,2018072401,CA,Fire,9,FOS,High,13.0,DHS-FPS,4382DRCAFPS01,DR,Activate FPS to DR-4382 to Contract security o...,"As directed by an in coordination with FEMA, F...",DR-4382-CA,Fire,2018-07-23 00:00:00+00:00,6.0,Fire,WILDFIRES AND HIGH WINDS,2018.0,7.0,23.0
66,2018072401,CA,Fire,9,FOS,High,10.0,EPA,4382DRCAEPA01,DR,"Activate ESF # 10 EPA to DR-4382 JFO, or other...","As directed by and in coordination with FEMA, ...",DR-4382-CA,Fire,2018-07-23 00:00:00+00:00,6.0,Fire,WILDFIRES AND HIGH WINDS,2018.0,7.0,23.0
136,2018072401,CA,Fire,9,FOS,Lifesaving,1.0,DOT,4382DRCADOT01,DR,"Activate ESF #1, DOT to the California Governo...","As directed by and in coordination with FEMA, ...",DR-4382-CA,Fire,2018-07-23 00:00:00+00:00,6.0,Fire,WILDFIRES AND HIGH WINDS,2018.0,7.0,23.0
152,2023121901,ME,Severe Storm,1,FOS,Normal,7.0,GSA,4754DRMEGSA03,DR,GSA to assist with leasing of space needed to...,"As directed by and in coordination with FEMA, ...",DR-4754-ME,Severe Storm,2023-12-17 00:00:00+00:00,23.0,"Flood,Severe Storm",SEVERE STORM AND FLOODING,2023.0,12.0,17.0
153,2023121901,ME,Severe Storm,1,FOS,High,7.0,GSA,4754DRMEGSA02,DR,Activate GSA to assist with leasing of space n...,"As directed by and in coordination with FEMA, ...",DR-4754-ME,Severe Storm,2023-12-17 00:00:00+00:00,23.0,"Flood,Severe Storm",SEVERE STORM AND FLOODING,2023.0,12.0,17.0


Transform the training data set using approach that is suitable for multiple categorical feature columns and a multilabel target. We return the transformers so that the test data set can be transformed later.

In [19]:
feature_columns = [
    'incidentId',
    'state',
    'designatedIncidentTypes',
    'declarationType',
    'region',
    'year',
]

target_column = 'supportFunction'

X_train, y_train = create_set(df, feature_columns, target_column)

print(X_train.shape, len(y_train))
print(y_train[:5])
X_train.head(5)

(485, 6) 485
0    [1.0, 3.0, 4.0, 7.0, 8.0, 10.0, 11.0, 12.0, 15.0]
1                                                [4.0]
2                                                [7.0]
3                                                [2.0]
4                                                [7.0]
Name: supportFunction, dtype: object


,incidentId,state,designatedIncidentTypes,declarationType,region,year
0,2012102505,CT,Hurricane,DR,1,2012.0
1,2012102505,DC,Hurricane,EM,3,2012.0
2,2012102505,DE,Hurricane,DR,3,2012.0
3,2012102505,DE,Hurricane,EM,3,2012.0
4,2012102505,MD,Hurricane,DR,3,2012.0


In [20]:
# Split the string in designatedIncidentTypes to a list of strings.
# This cannot be done earlier due to groupby issues.
X_train['designatedIncidentTypes'] = X_train['designatedIncidentTypes'].str.split(',')


In [21]:
# Since designatedIncidentTypes is a list it must be transformed differently than the other
# feature columns.
feature_columns_transform = [
    'state',
    'declarationType',
    'region',
]

X_train_transformed, y_train_transformed, preprocessor_X, preprocessor_y = transform_x_y_train(
    X_train,
    y_train,
    feature_columns_transform,
    ['designatedIncidentTypes'],
    target_column
)

In [22]:
# Assign the year values to a variable, then drop year and incidentId from X_train_transformed.
groups_by_year = X_train_transformed['year'].values
X_train_transformed.drop(columns=['incidentId', 'year'], inplace=True)
print(X_train_transformed.shape)
X_train_transformed.head()

(485, 88)


,state_AK,state_AL,state_AR,state_AS,state_AZ,state_CA,state_CO,state_CT,state_DC,state_DE,state_FL,state_GA,state_GU,state_HI,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_MP,state_MS,state_MT,state_NC,state_ND,state_NE,state_NH,state_NJ,state_NM,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_PR,state_RI,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_VI,state_VT,state_WA,state_WI,state_WV,state_WY,declarationType_DR,declarationType_EM,region_1,region_2,region_3,region_4,region_5,region_6,region_7,region_8,region_9,region_10,designatedIncidentTypes_Biological,designatedIncidentTypes_Chemical,designatedIncidentTypes_Coastal Storm,designatedIncidentTypes_Dam/Levee Break,designatedIncidentTypes_Earthquake,designatedIncidentTypes_Fire,designatedIncidentTypes_Flood,designatedIncidentTypes_Hurricane,designatedIncidentTypes_Mud/Landslide,designatedIncidentTypes_Other,designatedIncidentTypes_Severe Ice Storm,designatedIncidentTypes_Severe Storm,designatedIncidentTypes_Snowstorm,designatedIncidentTypes_Straight-Line Winds,designatedIncidentTypes_Terrorist,designatedIncidentTypes_Tornado,designatedIncidentTypes_Tropical Storm,designatedIncidentTypes_Typhoon,designatedIncidentTypes_Volcanic Eruption,designatedIncidentTypes_Winter Storm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
print(y_train_transformed.shape)
y_train_transformed.head()

(485, 16)


,ESF_0,ESF_1,ESF_2,ESF_3,ESF_4,ESF_5,ESF_6,ESF_7,ESF_8,ESF_9,ESF_10,ESF_11,ESF_12,ESF_13,ESF_14,ESF_15
0,0,1,0,1,1,0,0,1,1,0,1,1,1,0,0,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Run a variety of machine learning models and base classifiers using cross validation for model selection purposes.

In [45]:
base_svc = SVC(
        gamma='auto',
        decision_function_shape='ovo',
        class_weight='balanced',
        random_state=random_state,
        )
base_xgb = XGBClassifier(
        max_depth=12,
        random_state=0,
        n_estimators=100,
        base_score=0.5,
        )
base_gnb = GaussianNB()

br_classifier_svc = BinaryRelevance(
    classifier = base_svc,
    require_dense = [True, True]
)
br_classifier_xgb = BinaryRelevance(
    classifier = base_xgb,
    require_dense = [True, True]
)
br_classifier_gnb = BinaryRelevance(
    classifier = base_gnb,
    require_dense = [True, True]
)

cc_classifier_svc = ClassifierChain(
    classifier=base_svc, 
    require_dense=[True, True],
    order = list(range(0,16)),
)
cc_classifier_xgb = ClassifierChain(
    classifier=base_xgb,
    require_dense=[True, True],
    order = list(range(0,16)),
)
cc_classifier_gnb = ClassifierChain(
    classifier=base_gnb,
    require_dense=[True, True],
    order = list(range(0,16)),
)

classifier_rake_svc = RakelO(
   base_classifier=base_svc,
    base_classifier_require_dense=[True, True],
    labelset_size=3,
    model_count=32
)
classifier_rake_xgb = RakelO(
   base_classifier=base_xgb,
    base_classifier_require_dense=[True, True],
    labelset_size=3,
    model_count=32
)
classifier_rake_gnb = RakelO(
   base_classifier=base_gnb,
    base_classifier_require_dense=[True, True],
    labelset_size=3,
    model_count=32
)

moc_svc = MultiOutputClassifier(estimator=base_svc)
moc_xgb = MultiOutputClassifier(estimator=base_xgb)
moc_gnb = MultiOutputClassifier(estimator=base_gnb)

ovr_svc = OneVsRestClassifier(estimator=base_svc)
ovr_xgb = OneVsRestClassifier(estimator=base_xgb)
ovr_gnb = OneVsRestClassifier(estimator=base_gnb)

clf_list = [
    br_classifier_svc,
    br_classifier_xgb,
    br_classifier_gnb,
    cc_classifier_svc,
    cc_classifier_xgb,
    cc_classifier_gnb,
    classifier_rake_svc,
    classifier_rake_xgb,
    classifier_rake_gnb,
    moc_svc,
    moc_xgb,
    moc_gnb,
    ovr_svc,
    ovr_xgb,
    ovr_gnb,
]

hamming_scorer = make_scorer(
    hamming_loss,
    greater_is_better=False, 
)

F1_Macro_Avg = make_scorer(
    f1_score,
    average='macro', 
    zero_division=1,
)

F1_Samples_Avg = make_scorer(
    f1_score,
    average='macro', 
    zero_division=1,
)

scoring = {
    'Hamming':hamming_scorer,
    'F1_Macro_Avg':F1_Macro_Avg,
    'F1_Samples_Avg':F1_Samples_Avg,
    'jaccard_samples':'jaccard_samples',
}
import warnings
warnings.filterwarnings(
    "ignore", 
    category=UserWarning, 
    module="sklearn.utils.validation" 
)

for clf in clf_list:
    cross_val_results = cross_validate(clf,
                                       X_train_transformed,
                                       y_train_transformed,
                                       cv=10,
                                       scoring=scoring)

    print(f'{clf} Mean Hamming: {cross_val_results['test_Hamming'].mean()}, {cross_val_results['test_Hamming'].std() }')
    print(f'\nMean F1 Macro Avg: {cross_val_results['test_F1_Macro_Avg'].mean()}, {cross_val_results['test_F1_Macro_Avg'].std() }')
    print(f'\nMean F1 Samples Avg: {cross_val_results['test_F1_Samples_Avg'].mean()}, {cross_val_results['test_F1_Samples_Avg'].std() }')
    print(f'\nMean Jaccard Samples: {cross_val_results['test_jaccard_samples'].mean()}, {cross_val_results['test_jaccard_samples'].std() }')

BinaryRelevance(classifier=SVC(class_weight='balanced',
                               decision_function_shape='ovo', gamma='auto',
                               random_state=42),
                require_dense=[True, True]) Mean Hamming: -0.2947013180272109, 0.07151390028165508

Mean F1 Macro Avg: 0.46502319482358123, 0.07878793842017423

Mean F1 Samples Avg: 0.46502319482358123, 0.07878793842017423

Mean Jaccard Samples: 0.419884754205545, 0.04546304942241786
BinaryRelevance(classifier=XGBClassifier(base_score=0.5, booster=None,
                                         callbacks=None, colsample_bylevel=None,
                                         colsample_bynode=None,
                                         colsample_bytree=None, device=None,
                                         early_stopping_rounds=None,
                                         enable_categorical=False,
                                         eval_metric=None, feature_types=None,
                          

XGBoost appears to be our best base model. The most promising models appear to be BinaryRelevance, ClassifierChain, and OneVsRestClassifier (MultiOutputClassification had similar results and OneVsRestClassifier is more suited to this sort of task per documentation).

We can start with hyperparameter tuning of BinaryRelevance and the XGBoost base classifier.

In [ ]:
def objective_brxgb(trial, X, y, groups, cv_args):

    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 7, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 3, 10, log=True),
        'base_score': 0.5, 
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0,
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }

    
    base_XGB = XGBClassifier(**xgb_params)
    
    # The RakelO multilabel classifier (the main estimator)
    clf = BinaryRelevance(
        classifier = base_XGB,
        require_dense = [True, True]
    )

    scores = []    
    # cv = GroupTimeSeriesSplit(**cv_args)
    cv = KFold(**cv_args)

    # for train_index, val_index in cv.split(X, y, groups=groups):
    for train_index, val_index in cv.split(X, y):
    
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        
        # hl = hamming_loss(y_val, y_pred)
        # hl = f1_score(y_val, y_pred, average='samples', zero_division=1)
        eval_scorer = jaccard_score(y_val, y_pred, average='samples')

        scores.append(eval_scorer)

    return np.mean(scores)

cv_args = {'n_splits': 5, 'shuffle': True, 'random_state': 42}

objective_brxgb = partial(objective_brxgb,
                        X=X_train_transformed,
                        y=y_train_transformed,
                        groups=groups_by_year,
                        cv_args=cv_args,
                        )

# 3. Create and Run the Optuna Study
study_brxgb = optuna.create_study(direction="maximize") # Maximize the F1 score

study_brxgb.optimize(objective_brxgb, n_trials=30, show_progress_bar=True)

# --- Final Results ---
print("Optuna Hyperparameter Tuning Complete")
print(f"Best Score (Validation): {study_brxgb.best_value:.4f}")
print("Best Parameters:")
for key, value in study_brxgb.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 09:27:56,865] A new study created in memory with name: no-name-c93afa28-1b27-4625-b005-efe3ff505eb6
Best trial: 0. Best value: 0.427795:   3%|▎         | 1/30 [00:14<07:10, 14.86s/it]

[I 2025-11-15 09:28:11,725] Trial 0 finished with value: 0.42779457369663554 and parameters: {'n_estimators': 125, 'max_depth': 13, 'learning_rate': 0.003338318161433101, 'subsample': 0.8755838464745813, 'colsample_bytree': 0.8469636023590528, 'scale_pos_weight': 7.048015533411313}. Best is trial 0 with value: 0.42779457369663554.


Best trial: 1. Best value: 0.443922:   7%|▋         | 2/30 [00:31<07:31, 16.11s/it]

[I 2025-11-15 09:28:28,711] Trial 1 finished with value: 0.44392182273625574 and parameters: {'n_estimators': 150, 'max_depth': 13, 'learning_rate': 0.0815720542117103, 'subsample': 0.9357231531084991, 'colsample_bytree': 0.9250184631603144, 'scale_pos_weight': 6.453125247668684}. Best is trial 1 with value: 0.44392182273625574.


Best trial: 1. Best value: 0.443922:  10%|█         | 3/30 [00:45<06:48, 15.15s/it]

...

[I 2025-11-15 09:35:23,446] Trial 27 finished with value: 0.4411254982131271 and parameters: {'n_estimators': 175, 'max_depth': 13, 'learning_rate': 0.007886555482516848, 'subsample': 0.7131740083680101, 'colsample_bytree': 0.8803905891653233, 'scale_pos_weight': 4.720402451359302}. Best is trial 22 with value: 0.45380828793199923.


Best trial: 22. Best value: 0.453808:  97%|█████████▋| 29/30 [07:40<00:16, 16.72s/it]

[I 2025-11-15 09:35:37,372] Trial 28 finished with value: 0.45250454013340613 and parameters: {'n_estimators': 200, 'max_depth': 14, 'learning_rate': 0.015021823067354723, 'subsample': 0.670845352354594, 'colsample_bytree': 0.9472069596239205, 'scale_pos_weight': 3.559633241458858}. Best is trial 22 with value: 0.45380828793199923.


Best trial: 22. Best value: 0.453808: 100%|██████████| 30/30 [08:02<00:00, 16.07s/it]

[I 2025-11-15 09:35:58,961] Trial 29 finished with value: 0.4358010631979704 and parameters: {'n_estimators': 175, 'max_depth': 13, 'learning_rate': 0.016210966611190477, 'subsample': 0.6692953387792109, 'colsample_bytree': 0.8510348399125045, 'scale_pos_weight': 8.530394916520034}. Best is trial 22 with value: 0.45380828793199923.
Optuna Hyperparameter Tuning Complete
Best Hamming Score (Validation): 0.4538
Best Parameters:
  n_estimators: 200
  max_depth: 14
  learning_rate: 0.0012605477994259533
  subsample: 0.6426730736617045
  colsample_bytree: 0.9488151543720982
  scale_pos_weight: 3.015500783322584


In [58]:
study_brxgb_best_params = study_brxgb.best_params

study_brxgb_best_params_xgb = {
    'n_estimators': study_brxgb_best_params['n_estimators'],
    'max_depth': study_brxgb_best_params['max_depth'],
    'learning_rate': study_brxgb_best_params['learning_rate'],
    'subsample': study_brxgb_best_params['subsample'],
    'colsample_bytree' : study_brxgb_best_params['colsample_bytree'],
    'scale_pos_weight': study_brxgb_best_params['scale_pos_weight'],
    'base_score': 0.5,
    'random_state': 42,
    'verbosity': 0,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

study_brxgb_base_xgb = XGBClassifier(**study_brxgb_best_params_xgb)
study_brxgb_tuned = BinaryRelevance(
    classifier = study_brxgb_base_xgb,
    require_dense = [True, True]
)

study_brxgb_tuned.fit(X_train_transformed, y_train_transformed)

y_pred_training_br = study_brxgb_tuned.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_br.toarray())

display_error_heatmap(y_train_transformed, y_pred_training_br)

display_label_cardinality_error(y_train_transformed, y_pred_training_br.toarray())

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      1      0      1      1      0      0      1      1      0   
1      0      0      0      0      1      0      0      0      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      0      0      0      0      0      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       1       1       1       0       0       1  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       0       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[0 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1]
 [0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



We see some rows with many false positives in the error heatmap, and we see that many of our predictions include too many labels and others too few.

The ClassifierChain we started with used a base order for the labels. We can try to improve the outcome using correlation-based ordering.

In [62]:
# 1. Ensure the correlation matrix has integer indices/columns
if isinstance(y_train_transformed, np.ndarray):
    # If starting from NumPy, force integer columns
    Y_df = pd.DataFrame(y_train_transformed, columns=list(range(y_train_transformed.shape[1])))
else:
    # If starting from a DataFrame, reset the columns to integers
    Y_df = y_train_transformed.copy()
    Y_df.columns = list(range(Y_df.shape[1]))
    
correlation_matrix = Y_df.corr() 

num_labels = correlation_matrix.shape[0]
label_indices = list(range(num_labels))

In [63]:
total_correlations = correlation_matrix.abs().sum(axis=1)

start_label = total_correlations.idxmax()

ordered_labels = [start_label]
remaining_labels = set(label_indices) - {start_label}

In [64]:
while remaining_labels:
    best_next_label = -1
    max_avg_corr = -1

    for candidate_label in remaining_labels:
        # Calculate the absolute correlation between the candidate and all chosen labels
        corrs_with_chosen = correlation_matrix.loc[candidate_label, ordered_labels].abs()
        
        # Calculate the average (or sum) of these correlations
        avg_corr = corrs_with_chosen.mean() 

        if avg_corr > max_avg_corr:
            max_avg_corr = avg_corr
            best_next_label = candidate_label

    # Add the best label to the order and remove it from the remaining set
    if best_next_label != -1:
        ordered_labels.append(best_next_label)
        remaining_labels.remove(best_next_label)
    else:
        # Should not happen if the correlation matrix is correctly defined
        break

# The final result is the desired order for the ClassifierChain
final_chain_order = ordered_labels
print(f"Optimal ClassifierChain Order: {final_chain_order}")

Optimal ClassifierChain Order: [np.int64(1), 10, 12, 11, 2, 4, 9, 6, 8, 5, 13, 3, 15, 0, 14, 7]


In [65]:
def objective_ccxgb(trial, X, y, groups, cv_args, final_chain_order):

    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 7, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 3, 10, log=True),
        'base_score': 0.5, 
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0,
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }
        
    base_XGB = XGBClassifier(**xgb_params)
    
    # The RakelO multilabel classifier (the main estimator)
    clf = ClassifierChain(
        classifier = base_XGB,
        require_dense = [True, True],
        order = final_chain_order,
    )

    scores = []    
    # cv = GroupTimeSeriesSplit(**cv_args)
    cv = KFold(**cv_args)

    # for train_index, val_index in cv.split(X, y, groups=groups):
    for train_index, val_index in cv.split(X, y):
    
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        
        # hl = hamming_loss(y_val, y_pred)
        # hl = f1_score(y_val, y_pred, average='samples', zero_division=1)
        eval_scorer = jaccard_score(y_val, y_pred, average='samples')

        scores.append(eval_scorer)

    return np.mean(scores)

cv_args = {'n_splits': 5, 'shuffle': True, 'random_state': 42}

objective_ccxgb = partial(objective_ccxgb,
                        X=X_train_transformed,
                        y=y_train_transformed,
                        groups=groups_by_year,
                        cv_args=cv_args,
                        final_chain_order=final_chain_order
                        )

# 3. Create and Run the Optuna Study
study_ccxgb = optuna.create_study(direction="maximize") # Maximize the F1 score

study_ccxgb.optimize(objective_ccxgb, n_trials=30, show_progress_bar=True)

# --- Final Results ---
print("Optuna Hyperparameter Tuning Complete")
print(f"Best Score (Validation): {study_ccxgb.best_value:.4f}")
print("Best Parameters:")
for key, value in study_ccxgb.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 10:01:54,659] A new study created in memory with name: no-name-e9cabcac-d8b2-4774-92e7-b01023052ffd
  0%|          | 0/30 [00:00<?, ?it/s]

Best trial: 0. Best value: 0.200225:   3%|▎         | 1/30 [00:18<09:05, 18.81s/it]

[I 2025-11-15 10:02:13,505] Trial 0 finished with value: 0.20022472659070595 and parameters: {'n_estimators': 150, 'max_depth': 12, 'learning_rate': 0.0012050526887609034, 'subsample': 0.6308573788745474, 'colsample_bytree': 0.8229864024057549, 'scale_pos_weight': 8.37541927571129}. Best is trial 0 with value: 0.20022472659070595.


Best trial: 1. Best value: 0.207042:   7%|▋         | 2/30 [00:27<06:03, 12.99s/it]

[I 2025-11-15 10:02:22,419] Trial 1 finished with value: 0.2070415609590867 and parameters: {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.012050444548013784, 'subsample': 0.7170248764116215, 'colsample_bytree': 0.66199422861704, 'scale_pos_weight': 7.880421543062396}. Best is trial 1 with value: 0.2070415609590867.


...

[I 2025-11-15 10:06:53,684] Trial 27 finished with value: 0.21293990373887278 and parameters: {'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.0015915628255517048, 'subsample': 0.8506304537737802, 'colsample_bytree': 0.6924521263483908, 'scale_pos_weight': 6.772087137933656}. Best is trial 6 with value: 0.22342068012171107.


Best trial: 6. Best value: 0.223421:  97%|█████████▋| 29/30 [05:09<00:15, 15.61s/it]

[I 2025-11-15 10:07:04,506] Trial 28 finished with value: 0.21576533146120774 and parameters: {'n_estimators': 125, 'max_depth': 7, 'learning_rate': 0.007186022618031249, 'subsample': 0.7883999759131618, 'colsample_bytree': 0.7876527486882147, 'scale_pos_weight': 8.065262184424816}. Best is trial 6 with value: 0.22342068012171107.


Best trial: 6. Best value: 0.223421: 100%|██████████| 30/30 [05:26<00:00, 10.89s/it]

[I 2025-11-15 10:07:21,319] Trial 29 finished with value: 0.21945472625884999 and parameters: {'n_estimators': 150, 'max_depth': 11, 'learning_rate': 0.0038449906880223157, 'subsample': 0.8871651816171002, 'colsample_bytree': 0.8199727897457674, 'scale_pos_weight': 9.08486916268178}. Best is trial 6 with value: 0.22342068012171107.
Optuna Hyperparameter Tuning Complete
Best Score (Validation): 0.2234
Best Parameters:
  n_estimators: 75
  max_depth: 7
  learning_rate: 0.0032147012500595446
  subsample: 0.9111185216446693
  colsample_bytree: 0.8708706820861241
  scale_pos_weight: 9.426670132864865


In [67]:
study_ccxgb_best_params = study_ccxgb.best_params

study_ccxgb_best_params_xgb = {
    'n_estimators': study_ccxgb_best_params['n_estimators'],
    'max_depth': study_ccxgb_best_params['max_depth'],
    'learning_rate': study_ccxgb_best_params['learning_rate'],
    'subsample': study_ccxgb_best_params['subsample'],
    'colsample_bytree' : study_ccxgb_best_params['colsample_bytree'],
    'scale_pos_weight': study_ccxgb_best_params['scale_pos_weight'],
    'base_score': 0.5,
    'random_state': 42,
    'verbosity': 0,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

study_ccxgb_bbase_xgb = XGBClassifier(**study_ccxgb_best_params_xgb)
study_ccxgb_tuned = ClassifierChain(
    classifier = study_ccxgb_bbase_xgb,
    require_dense = [True, True],
    order=final_chain_order,
)

study_ccxgb_tuned.fit(X_train_transformed, y_train_transformed)

y_pred_training_cc = study_ccxgb_tuned.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_cc.toarray())

display_error_heatmap(y_train_transformed, y_pred_training_cc.toarray())

display_label_cardinality_error(y_train_transformed, y_pred_training_cc.toarray())

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      1      0      1      1      0      0      1      1      0   
1      0      0      0      0      1      0      0      0      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      0      0      0      0      0      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       1       1       1       0       0       1  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       0       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 1. 1. 1. 0. 

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



It's certainly more aggressive in the number of labels it predicts than the first model, whic explains the high number of false positives in the error heatmap. We can try one instance using the standard order for comparison.

In [68]:
def objective_ccxgb2(trial, X, y, groups, cv_args, final_chain_order):

    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 7, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 3, 10, log=True),
        'base_score': 0.5, 
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0,
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }
        
    base_XGB = XGBClassifier(**xgb_params)
    
    # The RakelO multilabel classifier (the main estimator)
    clf = ClassifierChain(
        classifier = base_XGB,
        require_dense = [True, True],
        order = final_chain_order,
    )

    scores = []    
    # cv = GroupTimeSeriesSplit(**cv_args)
    cv = KFold(**cv_args)

    # for train_index, val_index in cv.split(X, y, groups=groups):
    for train_index, val_index in cv.split(X, y):
    
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        
        # hl = hamming_loss(y_val, y_pred)
        # hl = f1_score(y_val, y_pred, average='samples', zero_division=1)
        eval_scorer = jaccard_score(y_val, y_pred, average='samples')

        scores.append(eval_scorer)

    return np.mean(scores)

cv_args = {'n_splits': 5, 'shuffle': True, 'random_state': 42}

objective_ccxgb2 = partial(objective_ccxgb2,
                        X=X_train_transformed,
                        y=y_train_transformed,
                        groups=groups_by_year,
                        cv_args=cv_args,
                        final_chain_order=list(range(0,16)),
                        )

# 3. Create and Run the Optuna Study
study_ccxgb2 = optuna.create_study(direction="maximize") # Maximize the F1 score

study_ccxgb2.optimize(objective_ccxgb2, n_trials=30, show_progress_bar=True)

# --- Final Results ---
print("Optuna Hyperparameter Tuning Complete")
print(f"Best Score (Validation): {study_ccxgb2.best_value:.4f}")
print("Best Parameters:")
for key, value in study_ccxgb2.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 10:15:15,551] A new study created in memory with name: no-name-8e2c0b0e-7589-484f-b6b3-2e329ddf0457
Best trial: 0. Best value: 0.438911:   3%|▎         | 1/30 [00:10<05:10, 10.71s/it]

[I 2025-11-15 10:15:26,272] Trial 0 finished with value: 0.4389106483436381 and parameters: {'n_estimators': 75, 'max_depth': 12, 'learning_rate': 0.011768689623371508, 'subsample': 0.779234477793001, 'colsample_bytree': 0.6228869427061535, 'scale_pos_weight': 5.366408611478881}. Best is trial 0 with value: 0.4389106483436381.


Best trial: 0. Best value: 0.438911:   7%|▋         | 2/30 [00:19<04:35,  9.84s/it]

[I 2025-11-15 10:15:35,492] Trial 1 finished with value: 0.4137056471592554 and parameters: {'n_estimators': 100, 'max_depth': 11, 'learning_rate': 0.007656667312091308, 'subsample': 0.624955744881925, 'colsample_bytree': 0.8770597154950144, 'scale_pos_weight': 9.707442656538566}. Best is trial 0 with value: 0.4389106483436381.


Best trial: 2. Best value: 0.440484:  10%|█         | 3/30 [00:29<04:28,  9.94s/it]

...

[I 2025-11-15 10:20:04,064] Trial 27 finished with value: 0.4438775800373739 and parameters: {'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.004419624899407971, 'subsample': 0.627879564419932, 'colsample_bytree': 0.7115452291095897, 'scale_pos_weight': 4.046423633965064}. Best is trial 15 with value: 0.4521210461932112.


Best trial: 15. Best value: 0.452121:  97%|█████████▋| 29/30 [05:02<00:11, 11.86s/it]

[I 2025-11-15 10:20:17,637] Trial 28 finished with value: 0.4453088120356162 and parameters: {'n_estimators': 200, 'max_depth': 9, 'learning_rate': 0.0022922094336784503, 'subsample': 0.6715664975688206, 'colsample_bytree': 0.8475804064089858, 'scale_pos_weight': 4.952510777231797}. Best is trial 15 with value: 0.4521210461932112.


Best trial: 15. Best value: 0.452121: 100%|██████████| 30/30 [05:17<00:00, 10.59s/it]

[I 2025-11-15 10:20:33,346] Trial 29 finished with value: 0.43898083016639716 and parameters: {'n_estimators': 125, 'max_depth': 12, 'learning_rate': 0.011923506954362896, 'subsample': 0.604075605676646, 'colsample_bytree': 0.622436291350757, 'scale_pos_weight': 5.835805661616327}. Best is trial 15 with value: 0.4521210461932112.
Optuna Hyperparameter Tuning Complete
Best Score (Validation): 0.4521
Best Parameters:
  n_estimators: 150
  max_depth: 11
  learning_rate: 0.002066833925524604
  subsample: 0.63534384488379
  colsample_bytree: 0.7733226597180934
  scale_pos_weight: 3.085892228599734


In [69]:
study_ccxgb2_best_params = study_ccxgb2.best_params

study_ccxgb2_best_params_xgb = {
    'n_estimators': study_ccxgb2_best_params['n_estimators'],
    'max_depth': study_ccxgb2_best_params['max_depth'],
    'learning_rate': study_ccxgb2_best_params['learning_rate'],
    'subsample': study_ccxgb2_best_params['subsample'],
    'colsample_bytree' : study_ccxgb2_best_params['colsample_bytree'],
    'scale_pos_weight': study_ccxgb2_best_params['scale_pos_weight'],
    'base_score': 0.5,
    'random_state': 42,
    'verbosity': 0,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

study_ccxgb2_bbase_xgb = XGBClassifier(**study_ccxgb2_best_params_xgb)
study_ccxgb2_tuned = ClassifierChain(
    classifier = study_ccxgb2_bbase_xgb,
    require_dense = [True, True],
    order=list(range(0,16)),
)

study_ccxgb2_tuned.fit(X_train_transformed, y_train_transformed)

y_pred_training_cc2 = study_ccxgb2_tuned.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_cc2.toarray())

display_error_heatmap(y_train_transformed, y_pred_training_cc2.toarray())

display_label_cardinality_error(y_train_transformed, y_pred_training_cc2.toarray())

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      1      0      1      1      0      0      1      1      0   
1      0      0      0      0      1      0      0      0      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      0      0      0      0      0      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       1       1       1       0       0       1  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       0       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 1. 

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



The metrics are improved but the model is still aggressive in predicting false positives. Our final hypertuning instance will be for OneVsRest.

Binary Relevance is a decent baseline. The next model up is Classifier Chain, which should perform better if set up correctly.

In [70]:
def objective_ovrxgb(trial, X, y, groups, cv_args):

    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 7, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 3, 10, log=True),
        'base_score': 0.5, 
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0,
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }
        
    base_XGB = XGBClassifier(**xgb_params)
    
    # The RakelO multilabel classifier (the main estimator)
    clf = OneVsRestClassifier(
        estimator = base_XGB,
    )

    scores = []    
    # cv = GroupTimeSeriesSplit(**cv_args)
    cv = KFold(**cv_args)

    # for train_index, val_index in cv.split(X, y, groups=groups):
    for train_index, val_index in cv.split(X, y):
    
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        
        # hl = hamming_loss(y_val, y_pred)
        # hl = f1_score(y_val, y_pred, average='samples', zero_division=1)
        eval_scorer = jaccard_score(y_val, y_pred, average='samples')

        scores.append(eval_scorer)

    return np.mean(scores)

cv_args = {'n_splits': 5, 'shuffle': True, 'random_state': 42}

objective_ovrxgb = partial(objective_ovrxgb,
                        X=X_train_transformed,
                        y=y_train_transformed,
                        groups=groups_by_year,
                        cv_args=cv_args,
                        )

# 3. Create and Run the Optuna Study
study_ovrxgb = optuna.create_study(direction="maximize") # Maximize the F1 score

study_ovrxgb.optimize(objective_ovrxgb, n_trials=30, show_progress_bar=True)

# --- Final Results ---
print("Optuna Hyperparameter Tuning Complete")
print(f"Best Score (Validation): {study_ovrxgb.best_value:.4f}")
print("Best Parameters:")
for key, value in study_ovrxgb.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 10:25:42,334] A new study created in memory with name: no-name-f38abe6d-b8e1-4fbf-a334-7f457b198fa0
Best trial: 0. Best value: 0.44415:   3%|▎         | 1/30 [00:06<03:09,  6.52s/it]

[I 2025-11-15 10:25:48,861] Trial 0 finished with value: 0.4441500481191203 and parameters: {'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.0015269653185780267, 'subsample': 0.7953481884092196, 'colsample_bytree': 0.7340171428424385, 'scale_pos_weight': 3.9684516805460275}. Best is trial 0 with value: 0.4441500481191203.


Best trial: 0. Best value: 0.44415:   7%|▋         | 2/30 [00:15<03:35,  7.70s/it]

[I 2025-11-15 10:25:57,386] Trial 1 finished with value: 0.43533792095647766 and parameters: {'n_estimators': 50, 'max_depth': 14, 'learning_rate': 0.09507213218913584, 'subsample': 0.6257613667477866, 'colsample_bytree': 0.6267882317303097, 'scale_pos_weight': 8.819236418063925}. Best is trial 0 with value: 0.4441500481191203.


Best trial: 0. Best value: 0.44415:  10%|█         | 3/30 [00:35<06:03, 13.46s/it]

...

[I 2025-11-15 10:32:29,947] Trial 27 finished with value: 0.43193349033555223 and parameters: {'n_estimators': 125, 'max_depth': 8, 'learning_rate': 0.06911676536656539, 'subsample': 0.9234464134783458, 'colsample_bytree': 0.8533362653687142, 'scale_pos_weight': 7.303933791919177}. Best is trial 23 with value: 0.45346835009206143.


Best trial: 23. Best value: 0.453468:  97%|█████████▋| 29/30 [07:00<00:13, 13.40s/it]

[I 2025-11-15 10:32:42,432] Trial 28 finished with value: 0.4412472893658461 and parameters: {'n_estimators': 100, 'max_depth': 14, 'learning_rate': 0.03371881678866989, 'subsample': 0.8429285233555964, 'colsample_bytree': 0.7180061891985303, 'scale_pos_weight': 4.347664908151131}. Best is trial 23 with value: 0.45346835009206143.


Best trial: 23. Best value: 0.453468: 100%|██████████| 30/30 [07:10<00:00, 14.34s/it]

[I 2025-11-15 10:32:52,630] Trial 29 finished with value: 0.44066064233590013 and parameters: {'n_estimators': 75, 'max_depth': 11, 'learning_rate': 0.0011604337792121117, 'subsample': 0.7962969097720464, 'colsample_bytree': 0.9457395287192616, 'scale_pos_weight': 3.7473647918937045}. Best is trial 23 with value: 0.45346835009206143.
Optuna Hyperparameter Tuning Complete
Best Score (Validation): 0.4535
Best Parameters:
  n_estimators: 100
  max_depth: 15
  learning_rate: 0.03356860978167583
  subsample: 0.8243841736622988
  colsample_bytree: 0.7587250324606077
  scale_pos_weight: 3.0505474642881847


In [72]:
study_ovrxgb_best_params = study_ovrxgb.best_params

study_ovrxgb_best_params_xgb = {
    'n_estimators': study_ovrxgb_best_params['n_estimators'],
    'max_depth': study_ovrxgb_best_params['max_depth'],
    'learning_rate': study_ovrxgb_best_params['learning_rate'],
    'subsample': study_ovrxgb_best_params['subsample'],
    'colsample_bytree' : study_ovrxgb_best_params['colsample_bytree'],
    'scale_pos_weight': study_ovrxgb_best_params['scale_pos_weight'],
    'base_score': 0.5,
    'random_state': 42,
    'verbosity': 0,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

study_ovrxgb_bbase_xgb = XGBClassifier(**study_ovrxgb_best_params_xgb)
study_ovrxgb_tuned = OneVsRestClassifier(
    estimator = study_ovrxgb_bbase_xgb,
)

study_ovrxgb_tuned.fit(X_train_transformed, y_train_transformed)

y_pred_training_ovr = study_ovrxgb_tuned.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_ovr)

display_error_heatmap(y_train_transformed, y_pred_training_ovr)

display_label_cardinality_error(y_train_transformed, y_pred_training_ovr)

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      1      0      1      1      0      0      1      1      0   
1      0      0      0      0      1      0      0      0      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      0      0      0      0      0      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       1       1       1       0       0       1  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       0       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1]
 [0 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



OneVsRest is still relatively aggressive in terms of false positive but does have the best overall metrics score and an impressive f1_macro score.

We will run one quick check of the untuned classifier for comparison.

In [73]:
ovr_xgb.fit(X_train_transformed, y_train_transformed)

y_pred_training_ovr_base = ovr_xgb.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_ovr_base)

display_error_heatmap(y_train_transformed, y_pred_training_ovr_base)

display_label_cardinality_error(y_train_transformed, y_pred_training_ovr_base)

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      1      0      1      1      0      0      1      1      0   
1      0      0      0      0      1      0      0      0      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      0      0      0      0      0      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       1       1       1       0       0       1  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       0       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



This has fewer false positive but seems more prone to underestimate the number of labels than our tuned model. Since we hope to provide more options for our end user to select, we will move forward with our tuned model even though this has a higher combined metrics score.

We can now load in the test set to measure performance of our tuned model.

In [89]:
df_test = pd.read_parquet('combined_test_set.parquet')
X_test, y_test = create_set(df_test, feature_columns, target_column)

In [90]:
# Split the string in designatedIncidentTypes to a list of strings.
# This cannot be done earlier due to groupby issues.
X_test['designatedIncidentTypes'] = X_test['designatedIncidentTypes'].str.split(',')
X_test.head()

,incidentId,state,designatedIncidentTypes,declarationType,region,year
0,2024010901,RI,[Severe Storm],DR,1,2024.0
1,2024020701,OR,"[Mud/Landslide, Straight-Line Winds, Winter St...",DR,10,2024.0
2,2024020801,CA,"[Flood, Severe Storm]",DR,9,2024.0
3,2024032101,OH,[Tornado],DR,5,2024.0
4,2024032102,WA,"[Mud/Landslide, Straight-Line Winds, Winter St...",DR,10,2024.0


In [91]:
y_test.head()

0               [7.0]
1          [0.0, 7.0]
2    [13.0, 3.0, 5.0]
3         [13.0, 7.0]
4               [7.0]
Name: supportFunction, dtype: object

In [ ]:
X_test_transformed, y_test_transformed = transform_x_y_test(
    X_test,
    y_test,
    preprocessor_X,
    preprocessor_y
)
X_test_transformed.drop(columns=['incidentId', 'year'], inplace=True)


/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/sklearn/preprocessing/_label.py:909: UserWarning:

unknown class(es) ['Tropical Depression'] will be ignored



In [93]:
print(X_test_transformed.shape)
X_test_transformed.head()

(80, 88)


,state_AK,state_AL,state_AR,state_AS,state_AZ,state_CA,state_CO,state_CT,state_DC,state_DE,state_FL,state_GA,state_GU,state_HI,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_MP,state_MS,state_MT,state_NC,state_ND,state_NE,state_NH,state_NJ,state_NM,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_PR,state_RI,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_VI,state_VT,state_WA,state_WI,state_WV,state_WY,declarationType_DR,declarationType_EM,region_1,region_2,region_3,region_4,region_5,region_6,region_7,region_8,region_9,region_10,designatedIncidentTypes_Biological,designatedIncidentTypes_Chemical,designatedIncidentTypes_Coastal Storm,designatedIncidentTypes_Dam/Levee Break,designatedIncidentTypes_Earthquake,designatedIncidentTypes_Fire,designatedIncidentTypes_Flood,designatedIncidentTypes_Hurricane,designatedIncidentTypes_Mud/Landslide,designatedIncidentTypes_Other,designatedIncidentTypes_Severe Ice Storm,designatedIncidentTypes_Severe Storm,designatedIncidentTypes_Snowstorm,designatedIncidentTypes_Straight-Line Winds,designatedIncidentTypes_Terrorist,designatedIncidentTypes_Tornado,designatedIncidentTypes_Tropical Storm,designatedIncidentTypes_Typhoon,designatedIncidentTypes_Volcanic Eruption,designatedIncidentTypes_Winter Storm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [94]:
print(y_test_transformed.shape)
y_test_transformed.head()

(80, 16)


,ESF_0,ESF_1,ESF_2,ESF_3,ESF_4,ESF_5,ESF_6,ESF_7,ESF_8,ESF_9,ESF_10,ESF_11,ESF_12,ESF_13,ESF_14,ESF_15
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [95]:
y_pred_test_over = study_ovrxgb_tuned.predict(X_test_transformed)

display_results(y_test_transformed, y_pred_test_over)

display_error_heatmap(y_test_transformed, y_pred_test_over)

display_label_cardinality_error(y_test_transformed, y_pred_test_over)



True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      0      0      0      0      0      0      1      0      0   
1      1      0      0      0      0      0      0      1      0      0   
2      0      0      0      1      0      1      0      0      0      0   
3      0      0      0      0      0      0      0      1      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       0       0       0       0       0       0  
1       0       0       0       0       0       0  
2       0       0       0       1       0       0  
3       0       0       0       1       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



In [98]:
y_proba = study_ovrxgb_tuned.predict_proba(X_test_transformed)

y_proba_adjusted = (y_proba >= 0.33).astype(int)

In [99]:
y_proba_adjusted

array([[0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       ...,
       [1, 0, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], shape=(80, 16))

In [101]:

display_results(y_test_transformed, y_proba_adjusted)

display_error_heatmap(y_test_transformed, y_proba_adjusted)

display_label_cardinality_error(y_test_transformed, y_proba_adjusted)

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      0      0      0      0      0      0      1      0      0   
1      1      0      0      0      0      0      0      1      0      0   
2      0      0      0      1      0      1      0      0      0      0   
3      0      0      0      0      0      0      0      1      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       0       0       0       0       0       0  
1       0       0       0       0       0       0  
2       0       0       0       1       0       0  
3       0       0       0       1       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [1 0 0 1 1 0 1 1 0 0 1 0 0 0 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)

